In [6]:
import json
import os
from copy import deepcopy
from typing import List, Tuple

In [7]:
hypothesis = 'patch_attention_pair_bias_small'

In [8]:
base_configuration = {
    'hypothesis': hypothesis,
    'experiment': None,  # will change below
    'data': {
        'path': '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl',
        'use_pesto': True,
        'ablation_string': '111111111',
        'max_folds': None,
        'use_coordinates': True
    },
    'model': {
        'name': 'patch_attention',
        'kwargs': {
            'features_mlp_hidden_sizes': None,
            'features_mlp_dropout_rate': None,
            'output_mlp_hidden_sizes': None, 
            'output_mlp_dropout_rate': None,
            'attention_mlp_hidden_sizes': None, 
            'attention_mlp_dropout_rate': None,
            'activation': 'relu',
            'input_shape': (10, 9),
            'max_number_of_patches': 10,
            'attention_dimension': None,
            'pairs_channel_dimension': None,
            'num_heads': None,
            'use_pair_bias': None
        }
    },
    'compile': {
        'optimizer': {
            'name': 'adamw',
            'kwargs': {'learning_rate': 1e-3}
        },
        'loss': {
            'name': 'binary_cross_entropy',
            'kwargs': {}
        }
    },
    'fit': {
        'epochs': 200,
        'batch_size': 256,
        'verbose': 1,
        'n_early_stopping_epochs': 12
    }
}

In [9]:
def mlps_option_to_str(mlps_option: List[Tuple[int, int]]) -> str:
    res = f'{str(mlps_option[0][0])}-{str(mlps_option[0][1])}'
    return f'{len(mlps_option)}_{res}'

In [10]:
def option_to_name(option: dict) -> str:
    name = f'pairbias_{option["use_pair_bias"]}-dim_{option["attention_dimension"]}-heads_{option["num_heads"]}'
    return name

In [11]:
dropout = 0.25
dims = [32, 64, 128]
heads = [1, 4, 8]
use_pair_biases = [True, False] 

options = [
    {
        'features_mlp_hidden_sizes': [(dim * 4, dim)],
        'features_mlp_dropout_rate': dropout,
        'output_mlp_hidden_sizes': [(dim * 4, dim)], 
        'output_mlp_dropout_rate': dropout,
        'attention_mlp_hidden_sizes': [(dim * 4, dim)], 
        'attention_mlp_dropout_rate': dropout,
        'attention_dimension': dim,
        'pairs_channel_dimension': dim,
        'num_heads': num_heads,
        'use_pair_bias': use_pair_bias
    }
    for dim in dims
    for num_heads in heads
    for use_pair_bias in use_pair_biases
]

In [12]:
dir_path = f'../data/{hypothesis}'
os.makedirs(dir_path, exist_ok=True)

for option in options:
    configuration = deepcopy(base_configuration)
    configuration['model']['kwargs'].update(option)
    
    output_mlp_str = mlps_option_to_str(option['output_mlp_hidden_sizes'])
    attention_mlp_str = mlps_option_to_str(option['attention_mlp_hidden_sizes'])
    
    name = option_to_name(option)
    configuration['experiment'] = name
    
    with open(f'{dir_path}/{name}.json', 'w') as f:
        json.dump(configuration, f, indent=4)
    